In [4]:
import torch
import os




# GPU 할당 변경하기
GPU_NUM = 4  # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')

torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check



Current cuda device  4


In [5]:
import torch
import os

import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

path = '../data/'

In [6]:

def dB( magnitude ):

    eps = torch.finfo(float).eps
    decibel = 20*torch.log10( torch.abs(magnitude) + eps )
    return decibel




def scaler( L, R ):
    
    LR = torch.cat( [L,R], dim=0 )
    LR_cpu = LR.to('cpu')
    LR_npy = np.asarray(LR_cpu)
    
    z = StandardScaler().fit_transform(LR_npy[:])
    z = torch.as_tensor(z).to('cuda')
    z = z.reshape(2, 257, 382)

    return z[0], z[1]




def angle_to_digit( y_data ):
    
    for idx in range(y_data.shape[0]):
        
        y = y_data[idx]
        if y<0: y_data[idx] = 10
        else:   y_data[idx] = ( y//20 )
    
    return y_data


In [7]:
x_data = ['S_left_pt.pt', 'S_right_pt.pt',
          'S_left_phase_pt.pt','S_right_phase_pt.pt',
          'strchd_left.pt', 'strchd_right.pt',
          'strchd_left_phase.pt','strchd_right_phase.pt']

y_data = np.load(path+ 'angle.npy')
y_data = np.concatenate((y_data, y_data), 0)



S_left_1  = torch.load( path + x_data[0] ).to('cuda:5')
S_right_1 = torch.load( path + x_data[1] ).to('cuda:5')
S_left_phase_1  = torch.load( path + x_data[2] ).to('cuda:6')
S_right_phase_1 = torch.load( path + x_data[3] ).to('cuda:6')

S_left_2  = torch.load( path + x_data[4] ).to('cuda:5')
S_right_2 = torch.load( path + x_data[5] ).to('cuda:5')
S_left_phase_2  = torch.load( path + x_data[6] ).to('cuda:6')
S_right_phase_2 = torch.load( path + x_data[7] ).to('cuda:6')


S_left  = torch.cat([S_left_1,  S_left_2],  dim=0).to('cuda:5')
S_right = torch.cat([S_right_1, S_right_2], dim=0).to('cuda:5')
S_left_phase  = torch.cat([S_left_phase_1,  S_left_phase_2],  dim=0).to('cuda:6')
S_right_phase = torch.cat([S_right_phase_1, S_right_phase_2], dim=0).to('cuda:6')


In [8]:
print(S_left.shape)
print(y_data.shape)

torch.Size([2000, 257, 382])
(2000, 1)


In [9]:

x_data_cuda4 = torch.empty([1,4,257,382]).to('cuda:4')
for idx in range( 1000 ):
    
    
    mag_L = S_left[idx,:,:]
    mag_R = S_right[idx,:,:]
    phi_L = S_left_phase[idx,:,:]
    phi_R = S_right_phase[idx,:,:]


    """log[dB] scale"""
    log_L = dB( mag_L )
    log_R = dB( mag_R )


    """정규화"""
    #z_mag = scaler( mag_L, mag_R )
    z_mag = scaler( log_L, log_R )
    z_phi = scaler( phi_L, phi_R )
    
    
    x_left_mag ,  x_right_mag   =  z_mag
    x_left_phase, x_right_phase =  z_phi
    
    
    xl_m = x_left_mag.reshape(1,257,382)
    xl_p = x_left_phase.reshape(1,257,382)
    xr_m = x_right_mag.reshape(1,257,382)
    xr_p = x_right_phase.reshape(1,257,382)
    
    x_element = torch.cat([ xl_m, xr_m, xl_p, xr_p ], dim=0).to('cuda:4')
    x_element = x_element.reshape(1, 4, 257, 382)
    #(1, 4, 257, 382)
 
    
    x_data_cuda4 = torch.cat([x_data_cuda4, x_element], dim=0).to('cuda:4')

x_data_cuda4 = x_data_cuda4[1:]



In [10]:

x_data_cuda7 = torch.empty([1,4,257,382]).to('cuda:7')
for idx in range( 1000,2000 ):
    
    
    mag_L = S_left[idx,:,:]
    mag_R = S_right[idx,:,:]
    phi_L = S_left_phase[idx,:,:]
    phi_R = S_right_phase[idx,:,:]


    """log[dB] scale"""
    log_L = dB( mag_L )
    log_R = dB( mag_R )


    """정규화"""
    #z_mag = scaler( mag_L, mag_R )
    z_mag = scaler( log_L, log_R )
    z_phi = scaler( phi_L, phi_R )
    
    
    x_left_mag ,  x_right_mag   =  z_mag
    x_left_phase, x_right_phase =  z_phi
    
    
    xl_m = x_left_mag.reshape(1,257,382)
    xl_p = x_left_phase.reshape(1,257,382)
    xr_m = x_right_mag.reshape(1,257,382)
    xr_p = x_right_phase.reshape(1,257,382)
    
    x_element = torch.cat([ xl_m, xr_m, xl_p, xr_p ], dim=0).to('cuda:7')
    x_element = x_element.reshape(1, 4, 257, 382)
    #(1, 4, 257, 382)
 
    
    x_data_cuda7 = torch.cat([x_data_cuda7, x_element], dim=0).to('cuda:7')

x_data_cuda7 = x_data_cuda7[1:]



In [11]:
x_data1 = x_data_cuda4.to('cpu')
x_data2 = x_data_cuda7.to('cpu')

x_data = torch.cat([x_data1, x_data2], dim=0).to('cpu')
y_data = torch.as_tensor( angle_to_digit(y_data) ).reshape(1000*2,)

In [12]:
print(x_data.shape)
print(y_data.shape)

torch.Size([2000, 4, 257, 382])
torch.Size([2000])


In [13]:
torch.save( x_data, '0818data.pt' )
torch.save( y_data, '0818label.pt')